In [1]:
"""
To know more or get code samples, please visit my website:
https://morvanzhou.github.io/tutorials/
Or search: 莫烦Python
Thank you for supporting!
"""

# please note, all tutorial code are running under python3.5.
# If you use the version like python2.7, please modify the code accordingly

# 8 - RNN LSTM Regressor example

# to try tensorflow, un-comment following two lines
# import os
# os.environ['KERAS_BACKEND']='tensorflow'
%matplotlib inline
import numpy as np
np.random.seed(1337)  # for reproducibility
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import Adam

BATCH_START = 0
TIME_STEPS = 20
BATCH_SIZE = 50
INPUT_SIZE = 1
OUTPUT_SIZE = 1
CELL_SIZE = 20
LR = 0.006

Using TensorFlow backend.


In [ ]:
def get_batch():
    global BATCH_START, TIME_STEPS
    # xs shape (50batch, 20steps)
    xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
    seq = np.sin(xs)
    res = np.cos(xs)
    BATCH_START += TIME_STEPS
    # plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
    # plt.show()
    return [seq[:, :, np.newaxis], res[:, :, np.newaxis], xs]

In [ ]:
model = Sequential()
# build a LSTM RNN
model.add(LSTM(
    batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    output_dim=CELL_SIZE,
    return_sequences=True,      # True: output at all steps. False: output as last step.
    stateful=True,              # True: the final state of batch1 is feed into the initial state of batch2
))
# add output layer
model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
adam = Adam(LR)
model.compile(optimizer=adam, loss='mse',)

In [24]:
import pandas as pd
# alvin test code
# xs shape (50batch, 20steps)
print('BATCH_START: '+str(BATCH_START) +', TIME_STEPS: ' + str(TIME_STEPS) + ',BATCH_SIZE: '+str(BATCH_SIZE))
print('TIME_STEPS*BATCH_SIZE: ' + str(TIME_STEPS*BATCH_SIZE))

xs = np.arange(BATCH_START, BATCH_START+TIME_STEPS*BATCH_SIZE).reshape((BATCH_SIZE, TIME_STEPS)) / (10*np.pi)
seq = np.sin(xs)
res = np.cos(xs)
BATCH_START += TIME_STEPS
# plt.plot(xs[0, :], res[0, :], 'r', xs[0, :], seq[0, :], 'b--')
# plt.show()
x_batch = seq[:, :, np.newaxis]
y_batch = res[:, :, np.newaxis]
print('seq.shape = ' + str(seq.shape))
#display(pd.DataFrame(seq))

print(x_batch.shape)
#display(pd.DataFrame(x_batch))
print(y_batch.shape)
print(seq.shape)
print('\nfirst time step seq[0], shape is'+str(seq[0].shape)+'\n')
print(seq[0])
print('first batch, first time step, the shape will be' + str(y_batch[0].shape))
print(y_batch[0])

BATCH_START: 440, TIME_STEPS: 20,BATCH_SIZE: 50
TIME_STEPS*BATCH_SIZE: 1000
seq.shape = (50, 20)
(50, 20, 1)
(50, 20, 1)
(50, 20)

first time step seq[0], shape is(20,)

[ 0.99136214  0.99503397  0.99769772  0.99935067  0.99999115  0.99961851
  0.99823313  0.99583642  0.9924308   0.98801972  0.98260765  0.97620008
  0.96880349  0.96042538  0.95107424  0.94075955  0.92949174  0.91728224
  0.90414342  0.89008859]
first batch, first time step, the shape will be(20, 1)
[[ 0.13115301]
 [ 0.09953587]
 [ 0.06781788]
 [ 0.03603118]
 [ 0.00420798]
 [-0.02761948]
 [-0.05941896]
 [-0.09115825]
 [-0.12280517]
 [-0.15432768]
 [-0.18569384]
 [-0.21687187]
 [-0.24783017]
 [-0.2785374 ]
 [-0.30896243]
 [-0.33907444]
 [-0.36884293]
 [-0.39823773]
 [-0.42722907]
 [-0.45578757]]


In [ ]:
print('Training ------------')
for step in range(501):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch, Y_batch, xs = get_batch()
    cost = model.train_on_batch(X_batch, Y_batch)
    pred = model.predict(X_batch, BATCH_SIZE)
    plt.plot(xs[0, :], Y_batch[0].flatten(), 'r', xs[0, :], pred.flatten()[:TIME_STEPS], 'b--')
    plt.ylim((-1.2, 1.2))
    plt.draw()
    plt.pause(0.1)
    if step % 10 == 0:
        print('train cost: ', cost)